In [ ]:
#import libraries
import pandas as pd
import numpy as np

In [ ]:
#Read tsv file
dataFrame = pd.read_csv("version1/reducedusersrows.tsv", sep = "\t", header=0, encoding = "latin1")

In [ ]:
#Retreive user id and events columns
prop14Rows = dataFrame[["post_uniqueId","post_prop14"]].reset_index(drop=True)

In [ ]:
prop14Rows

In [ ]:
#converting these columns into string type
prop14Rows['post_uniqueId'] = prop14Rows['post_uniqueId'].astype(str)
prop14Rows['post_prop14'] = prop14Rows['post_prop14'].astype(str)

In [ ]:
#Sort the rows by user id
prop14RowsSort = prop14Rows.sort_values(by=['post_uniqueId'])
prop14RowssSort = prop14RowsSort.reset_index(drop=True)

In [ ]:
#Merge all the events performed by user into single row
prop14RowsGroup = prop14RowsSort.groupby(['post_uniqueId'])['post_prop14'].apply(lambda x: '/'.join(x.str.replace('https://www.dell.com/','').str.replace('http://www.dell.com/',''))).reset_index()

In [ ]:
prop14RowsGroup

In [ ]:
#create an empty list
visitList=[]

In [ ]:
#loop through each cell of prop14 column and remove duplicate events
for index, row in prop14RowsGroup.iterrows():
    templist = set(pd.Series(prop14RowsGroup['post_prop14'].iloc[index].split('/')).tolist())
    visitList.append(",".join(templist))
    #print(index)

In [ ]:
visitList

In [ ]:
#Assigning visit links which doesnt contain duplicates
prop14RowsGroup['post_prop14'] = visitList

In [ ]:
prop14RowsGroup['post_prop14']

In [ ]:
#import Counter from library
from collections import Counter

In [ ]:
#Get count of visited links
visitCount  = pd.Series(prop14RowsGroup['post_prop14'].str.replace('[\[\]\']','').str.split(',').map(Counter).sum())

In [ ]:
visitCount

In [ ]:
#Initialize threshold 12.5% shows accessories
thre1 = prop14RowsGroup.shape[0]/2 #50%
thre2 = prop14RowsGroup.shape[0]/8 #12.5%
#Filter using the threshold. All the values less than thre will be substitued witn nan values
visitCountThre = visitCount.where(visitCount<thre1)

In [ ]:
#Filtering the visits which have ben performed atleast by 25% of users and atmost 50%
visitCountThre = visitCountThre.where(visitCount>thre2)

In [ ]:
#Remove nan values
visitCountThreWithoutNull = visitCountThre.dropna()

In [ ]:
#Get all unique visited links after thre
visitedList = visitCountThreWithoutNull.index.tolist()

In [ ]:
#Count of visited links
len(visitedList)

In [ ]:
visitedList

In [ ]:
#Add the events as columns to dataframe
for i in visitedList:
    name = 'prop14_'+i
    prop14RowsGroup[name] = 0

In [ ]:
for index, row in prop14RowsGroup.iterrows():
    templist = pd.Series(prop14RowsGroup['post_prop14'].iloc[index].split(',')).tolist()
    print(index)
    commonlist = [value for value in visitedList if value in templist]
    for event in commonlist:
        colname = 'prop14_'+event
        prop14RowsGroup[colname].iloc[index] = 1

In [ ]:
prop14RowsGroup

In [ ]:
#Save dataframe to a tsv file
prop14RowsGroup.to_csv("version1/flattenprop14.tsv", sep="\t", index=False)

In [ ]:
#Read the data which is in tsv format which contains flatened events columns
dataFrame = pd.read_csv("version1/modeldata.tsv", sep='\t')

In [ ]:
dataFrame

In [ ]:
#Storing purchased columns
purchased = dataFrame['purchased']

In [ ]:
#dropping purchased column
dataFrame = dataFrame.drop('purchased',1)

In [ ]:
dataFrame

In [ ]:
#Adding flattened prop14 columns
for name in visitedList:
    colname = "prop14_"+name
    dataFrame[colname] = prop14RowsGroup[colname]

In [ ]:
#Adding purcased column
dataFrame['purchased'] = purchased

In [ ]:
dataFrame

In [ ]:
#Save dataframe to a tsv file.
#This file consists of event and prop14 flattened columns along with other required and derived columns.
#This data can be used to build a model
dataFrame.to_csv("version1/modeldata1.tsv", sep="\t", index=False)